<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcoteran/deeplearning/blob/master/notebooks/3.2_deepleaningintroduction_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marcoteran/deeplearning/blob/master/notebooks/3.2_deepleaningintroduction_dnn.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Abrir en Kaggle" title="Abrir y ejecutar en Kaggle"/></a>
  </td>
</table>

### Ejemplo de código
# Sesión 08: Proyecto Precio de vehículos usados
## Deep Learning y series de tiempo

**Name:** Marco Teran **E-mail:** marco.tulio.teran@gmail.com,
[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

<h1 id="problema">Contexto analítico y exploración de datos</h1>

El mercado de autos usados es reconocido por ser un sector economico muy competido con un centenar de compañias que luchan por hacerse con una porción de la torta. El precio de los autos se devalua año año debido a multiples factores y determinar el precio correcto es clave para las compañias para lograr competir en el mercado. En este caso se requiere implementar una red neural que permita determinar el valor más justo para los vehiculos dependiento de sus atributos.

Se cuenta con un dataset (Craiglist_Cars.csv) que serán cargados directamente a Colab.


In [ ]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import io
#from google.colab import files #Librería necesaria para interactuar con archivos en Colab

In [ ]:
#!mkdir data
#!wget -O data/Craiglist_Cars.csv https://github.com/marcoteran/deeplearning/raw/master/notebooks/data/Craiglist_Cars.csv

In [ ]:
#uploaded = files.upload()
#cars = pd.read_csv(io.StringIO(uploaded['Craiglist_Cars.csv'].decode('utf-8')), sep = ',' )
cars = pd.read_csv('data/Craiglist_Cars.csv', sep = ',' )

In [ ]:
cars.head(3)

# 1. Evalua la cantidad, tipo y completitud de las variables disponibles

En esta sección, se realiza una exploración básica del conjunto de datos "cars" utilizando algunas funciones de pandas.

Primero, se utiliza la función shape de pandas para imprimir la cantidad de filas y columnas en "cars". Esto proporciona una idea inicial de la magnitud del conjunto de datos.

Luego, se utiliza la función isnull para detectar la cantidad de valores nulos en cada columna de "cars". Se calcula el porcentaje de valores nulos para cada columna y se imprime en pantalla.

In [ ]:
print(cars.shape)
100*cars.isnull().sum()/cars.shape[0]

A continuación, se utiliza la función DataFrame de pandas para crear un nuevo dataframe llamado "types", que almacena los tipos de datos de cada columna en "cars". Esto ayuda a comprender mejor la estructura de los datos y cómo se deben manejar

In [ ]:
types = pd.DataFrame(cars.dtypes)
print(types.groupby(0).size())

Finalmente, se crea una lista llamada "categoricas" que almacena los nombres de las columnas que contienen datos categóricos (es decir, no numéricos). Se utiliza un bucle para recorrer cada columna categórica y se imprime en pantalla la cantidad de valores únicos en esa columna. Esto proporciona información adicional sobre la naturaleza de los datos y cómo se deben procesar para el análisis posterior.

In [ ]:
categoricas = types.index[types[0] == 'O'].values
for line in categoricas:
 print("La variable "+ line +" contiene:",str(len(cars[line].unique()))+" distinct values")

# Ingeniería de datos

Ahora se debe preparar la información para poder alimentar la red neuronal.

# 2. Implementa estrategías para tratar la información nula en las variables cuya tasa de nulos sea máximo el 10%

En esta sección, se realizan algunas tareas de limpieza y preparación de datos para el conjunto de datos "cars".

1. En primer lugar, se rellenan los valores faltantes en la columna "fuel" con el valor más común utilizando la función fillna de pandas. Este es un ejemplo de cómo manejar valores nulos o faltantes en el conjunto de datos.
2. Luego, se vuelven a imprimir las dimensiones de "cars" y se verifica si hay valores nulos en otras columnas.
3. Después, se rellenan los valores faltantes en las columnas "title_status", "transmission" y "manufacturer" con el valor más común utilizando la función fillna de pandas.

In [ ]:
cars["fuel"] = cars["fuel"].fillna(cars["fuel"].mode()[0])

In [ ]:
print(cars.shape)
100*cars.isnull().sum()/cars.shape[0]

In [ ]:
cars["title_status"] = cars["title_status"].fillna(cars["title_status"].mode()[0])
cars["transmission"] = cars["transmission"].fillna(cars["transmission"].mode()[0])
cars["manufacturer"] = cars["manufacturer"].fillna(cars["manufacturer"].mode()[0])
#Verificamos el cambio
100*cars.isnull().sum()/cars.shape[0]

Luego del procedimiento anterior se debe proceder a convertir las variables categoricas en variables numericas. Durante el curso implementamos un método de One Hot Encoding disponible en Scikit Learn. En este caso utilizaremos una funcionalidad embedida en Pandas denominada ["get_dummies"](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html):

* A continuación, se crea una copia del dataframe llamado "df" y se aplican técnicas de codificación de variables categóricas. Se crea una nueva columna para cada valor posible en cada variable categórica y se codifica como 1 si la observación tiene ese valor y 0 si no lo tiene. Esto se realiza utilizando la función get_dummies de pandas.
* Además, se eliminan las columnas que contienen la categoría "other" ya que no aportan ningún valor al conjunto de datos.
* Luego, se vuelven a imprimir las dimensiones de "df" y se imprime una vista previa del conjunto de datos.

In [ ]:
df= cars.copy()
for col in categoricas:
    df = pd.concat([df, (pd.get_dummies(df[col])).astype(int)], axis=1)
    df.drop(columns=[col],inplace=True)
# Al crear las variables dummies se crean varias columnas referentes a categorias
# 'other' que no aportan ningún valor al dataset por lo cual las eliminamos
df.drop('other', axis=1, inplace=True)
print(df.shape)
df.head(3)

A continuación, se utiliza la función DataFrame de pandas para crear un nuevo dataframe llamado "types", que almacena los tipos de datos de cada columna en "df".

In [ ]:
types = pd.DataFrame(df.dtypes)
print("Tipos de variables",types.groupby(0).size())

Después, se seleccionan las columnas numéricas del conjunto de datos y se eliminan las que corresponden a la variable objetivo ("price"). Esto se realiza utilizando la función set de Python para encontrar las columnas numéricas y luego la función list para convertir el resultado en una lista.

Por último, se crea un nuevo dataframe llamado "variables_consolidadas" que contiene solo las variables numéricas y se crea un nuevo dataframe llamado "objetivo" que contiene solo la variable objetivo ("price").

In [ ]:
numeric_columns = list(set(types.index[types[0] =="int64"].values) - set(["price"]))
variables_consolidadas = df[numeric_columns]
objetivo = df["price"] #Variable objetivo de nuestra regresion.

variables_consolidadas.head(3)

# 3. Separa el set de datos consolidados en 3 sets (entrenamiento, prueba y validación) de acuerdo con las recomendaciones vistas en el curso.


En estas líneas de código se utiliza la librería Scikit-learn para dividir los datos en conjuntos de entrenamiento, validación y prueba. Se utiliza la función "train_test_split" para crear los tres conjuntos a partir de las variables consolidadas (x) y la variable objetivo (y).
* Primero, se divide en 80% de entrenamiento y 20% de prueba.
* Luego, se divide el conjunto de entrenamiento en 90% para entrenamiento y 10% para validación.

In [ ]:
# Coloca tu código aquí
from sklearn.model_selection import train_test_split

In [ ]:
#80% train 20% test
x_train,x_test, y_train,y_test = train_test_split(variables_consolidadas,objetivo,test_size=0.2, random_state=2020)

In [ ]:
#90% train 10% val
x_train,x_val, y_train,y_val = train_test_split(x_train,y_train,test_size=0.1, random_state=2020)

Posteriormente, se utiliza la función "reshape" para cambiar la forma de las variables objetivo y poder utilizarlas en modelos de aprendizaje automático. 

In [ ]:
y_train=y_train.values.reshape(-1,1)
y_test=y_test.values.reshape(-1,1)
y_val=y_val.values.reshape(-1,1)

Finalmente, se imprimen las formas (shapes) de los conjuntos de entrenamiento, validación y prueba para verificar que se hayan creado correctamente.

In [ ]:
print("Shape of x_train:",x_train.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_test:",y_test.shape)
print("Shape of y_val:",y_val.shape)

<h1 id="arquitectura">Diseño, Entrenamiento y Evaluación de la RN</h1>

Una vez consolidado los sets de información de entrenamiento, validacion y pruebas ya podemos iniciar a modelar nuestra red neuronal con las siguientes consideraciones:
* Realiza la prueba con un par de arquitecturas iniciales.
* Evalua el desempeño de la red.
* Si el desempeño es bajo vuelve a la información y prueba estrategias de estandarización de la información.

# 4. Implementa una red neuronal cuyas pérdidas (MSE) con el set de prueba sea menor a 0.40.

Se importa la clase StandardScaler de la biblioteca sklearn.preprocessing, la cual es una técnica de preprocesamiento de datos que escala los datos para que tengan una media de cero y una desviación estándar de uno. Esto es importante porque ayuda a que el modelo pueda trabajar con variables en la misma escala, lo que puede mejorar su rendimiento.

In [ ]:
#Coloca tu código aquí
from sklearn.preprocessing import StandardScaler

Se crea una instancia de StandardScaler() y se llama al método fit() con los datos de entrenamiento x_train, para que el objeto pueda aprender los parámetros de escalamiento a partir de estos datos.

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)

Se utiliza el método transform() para escalar los conjuntos de datos de entrenamiento, validación y prueba x_train, x_val y x_test, respectivamente.

In [ ]:
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

Se repiten los pasos anteriores para la variable objetivo y_train, y_val y y_test.

In [ ]:
scaler1 = StandardScaler()
scaler1.fit(y_train)

In [ ]:
y_train_scaled = scaler1.transform(y_train)
y_val_scaled = scaler1.transform(y_val)
y_test_scaled = scaler1.transform(y_test)

Se importa la clase Sequential y los módulos Dense, Dropout de la biblioteca Keras. 

In [ ]:
#Coloca tu código aquí
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model

Luego, se crea una instancia de la clase Sequential y se añaden varias capas Dense con activación relu, una capa Dropout y una capa final con activación linear.

In [ ]:
model = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation = "linear"))

Se compila el modelo con el optimizador adam, la función de pérdida mse (mean squared error) y la métrica mean_absolute_error.

In [ ]:
model.compile(optimizer = "adam",loss="mse",metrics=["mean_absolute_error"])

Se imprime un resumen del modelo con la función summary().

In [ ]:
print(model.summary())

In [ ]:
#!pip install pydot
#!pip install graphviz
#!conda install -c anaconda pydot=1.2.3
#!conda install -c anaconda pyparsing=2.2.0
#!conda install GraphViz

import errno
import pydot
from tensorflow.keras.utils import plot_model

Muestra el diagrama del modelo en un archivo PNG

In [ ]:
plot_model(model, to_file='model.png',show_shapes=True)

Se entrena el modelo llamando al método fit() con los datos escalados y se guardan los resultados del entrenamiento en el objeto modelhistory. El modelo se entrena durante 50 épocas con un tamaño de lote de 1024. También se proporcionan los datos de validación para que se evalúe el rendimiento del modelo en cada época.

In [ ]:
modelhistory=model.fit(x_train_scaled,y_train_scaled, validation_data = (x_val_scaled,y_val_scaled),epochs=50, batch_size=1024)

Ahora realiza la evaluación del modelo con el set de entrenamiento.

In [ ]:
result = model.evaluate(x_test_scaled,y_test_scaled)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":", str(round(result[i],2)))

Si el modelo cumple con el requerimiento, se guarda con el siguiente comando:

In [ ]:
model.save('predictedprices.h5')

# 5. Realiza un gráfico que evidencia la evolución de la función de pérdidas a traves de las distintas épocas de entrenamiento.

In [ ]:
modelhistory.history

In [ ]:
#Coloca tu código aquí
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(13,6))
plt.plot(modelhistory.history['loss'])
plt.plot(modelhistory.history['val_loss'])
plt.title("Pérdidas del modelo con set de entrenamiento y pruebas por época")
plt.ylabel('MSE')
plt.xlabel('Épocas')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.show()

Trata de realizar predicciones con el modelo

In [ ]:
real=pd.DataFrame(y_train)
predic=model.predict(pd.DataFrame(x_train_scaled))
valores_reescalados = scaler1.inverse_transform(predic)
pred_escal =pd.DataFrame(valores_reescalados)
# Muestra los valores reales y las predicciones
for i in range(0,5):
	print("Real=%s, Prediccion=%s" % (real[0][i], pred_escal[0][i]))

___
¡Todo bien! ¡Es todo por hoy! 😀